<a href="https://colab.research.google.com/github/NicoleLund/flight_delay_prediction/blob/nrl_210817/data_manipulation_modeling/feature_assessment/feature_assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

feature_assessment.ipynb
-----

Written in Google Colab

by Nicole Lund

This workbook investigates the correlation between features in 2017 flight performance prior to building a model from the data.

In [25]:
# Import dependencies
import pandas as pd
import datetime
from datetime import timedelta

In [2]:
# Read the CSV file from AWS to Pandas Dataframe
url = "https://finalproject-3.s3.us-west-1.amazonaws.com/tus_final_df.csv"

df = pd.read_csv(url)

df.head()

,Unnamed: 0,origin_city,dest_city,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,day_of_week
0,0,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2017-01-01,DL,2066,TUS,ATL,14:30:00,1459.0,29.0,10.0,1509.0,2011.0,17.0,20:03:00,2028.0,25.0,0.0,0,0.0,213.0,209.0,182.0,1541.0,11.0,0.0,0.0,0.0,14.0,Sunday
1,1,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2017-01-02,DL,1127,TUS,ATL,06:00:00,637.0,37.0,10.0,647.0,1157.0,6.0,11:29:00,1203.0,34.0,0.0,0,0.0,209.0,206.0,190.0,1541.0,34.0,0.0,0.0,0.0,0.0,Monday
2,2,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2017-01-02,DL,2066,TUS,ATL,14:30:00,1447.0,17.0,10.0,1457.0,1943.0,9.0,20:05:00,1952.0,-13.0,0.0,0,0.0,215.0,185.0,166.0,1541.0,0.0,0.0,0.0,0.0,0.0,Monday
3,3,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2017-01-03,DL,1127,TUS,ATL,06:00:00,559.0,-1.0,29.0,628.0,1116.0,5.0,11:29:00,1121.0,-8.0,0.0,0,0.0,209.0,202.0,168.0,1541.0,0.0,0.0,0.0,0.0,0.0,Tuesday
4,4,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2017-01-03,DL,2066,TUS,ATL,14:30:00,1441.0,11.0,11.0,1452.0,1939.0,9.0,20:05:00,1948.0,-17.0,0.0,0,0.0,215.0,187.0,167.0,1541.0,0.0,0.0,0.0,0.0,0.0,Tuesday


In [7]:
# List all of the column headers
df.columns

Index(['Unnamed: 0', 'origin_city', 'dest_city', 'FL_DATE', 'OP_CARRIER',
       'OP_CARRIER_FL_NUM', 'ORIGIN', 'DEST', 'CRS_DEP_TIME', 'DEP_TIME',
       'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON', 'TAXI_IN',
       'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'CANCELLED',
       'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME',
       'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'CARRIER_DELAY',
       'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY',
       'day_of_week'],
      dtype='object')

In [21]:
# Identify row to verify calculations with
row = 0

In [55]:
# Verify DEP_DELAY/DEP_TIME calculation

print('Scheduled Departure Time')
print(df.CRS_DEP_TIME[row])
print(pd.to_datetime(df.CRS_DEP_TIME[row], format = '%H:%M:%S'))
print('')
print('Actual Departure Time')
print(pd.to_datetime(df.DEP_TIME[row], format = '%H%M'))
# print(pd.to_datetime(df.DEP_TIME[row], format = '%H%M').time())
# print(pd.to_datetime(df.DEP_TIME[row], format = '%H%M').strftime(format = '%H:%M:%S'))
print('')

dep_delay = pd.to_datetime(df.DEP_TIME[row], format = '%H%M') - pd.to_datetime(df.CRS_DEP_TIME[row], format = '%H:%M:%S')
print('Departure Delay')
print(dep_delay)
print(df.DEP_DELAY[row])

Scheduled Departure Time
14:30:00
1900-01-01 14:30:00

Actual Departure Time
1900-01-01 14:59:00

Departure Delay
0 days 00:29:00
29.0


In [59]:
# Verify WHEELS_OFF time
print('Departure time: ')
print(df.DEP_TIME[row])
print('Taxi time: ')
print(df.TAXI_OUT[row])
print('WHEELS OFF time:')
print(df.WHEELS_OFF[row])

Departure time: 
1459.0
Taxi time: 
10.0
WHEELS OFF time:
1509.0


In [64]:
# Verify WHEELS_ON time
print('Arrival time: ')
print(df.ARR_TIME[row])
print('Taxi time: ')
print(df.TAXI_IN[row])
print('WHEELS ON time:')
print(df.WHEELS_ON[row])

Arrival time: 
2028.0
Taxi time: 
17.0
WHEELS ON time:
2011.0


In [63]:
# Verify WHEELS_ON time
print('WHEELS OFF time:')
print(df.WHEELS_OFF[row])
print('AIR_TIME: ')
print(df.AIR_TIME[row])
print('hours')
print(182/60)
print('minutes')
print(182-3*60)
print('WHEELS_ON time: ')
print(df.WHEELS_ON[row])

WHEELS OFF time:
1509.0
AIR_TIME: 
182.0
hours
3.033333333333333
minutes
2
WHEELS_ON time: 
2011.0


In [22]:
# Verify ACTUAL_ELAPSED_TIME calculation
actual_elapsed_time = df.TAXI_OUT[row]+df.TAXI_IN[row]+df.AIR_TIME[row]
print(actual_elapsed_time)
print(df.ACTUAL_ELAPSED_TIME[row])

209.0
209.0


In [68]:
# Verify Arrival time
print('Departure time: ')
print(df.DEP_TIME[row])
print('Actual Elapsed Time: ')
print(df.ACTUAL_ELAPSED_TIME[row])
print('hours')
print(209/60)
print('minutes')
print(209-3*60)
print('Arrival time: ')
print(df.ARR_TIME[row])
print(df.origin_city[row])
print(df.dest_city[row])

Departure time: 
1459.0
Actual Elapsed Time: 
209.0
hours
3.4833333333333334
minutes
29
Arrival time: 
2028.0
TUCSON, ARIZONA, USA               
ATLANTA, GEORGIA, USA              


In [23]:
# Verify ARR_DELAY calculation
arr_delay = df.CARRIER_DELAY[row]+df.WEATHER_DELAY[row] + df.NAS_DELAY[row] + df.SECURITY_DELAY[row] + df.LATE_AIRCRAFT_DELAY[row]
print(arr_delay)
print(df.ARR_DELAY[row])

25.0
25.0


In [76]:
# Verify if ARR_DELAY is directly calculable from X values
print('Departure Time')
print(df.CRS_DEP_TIME[row])
print('Transit time')
print(df.DEP_DELAY[row] + df.TAXI_OUT[row] + df.AIR_TIME[row] + df.TAXI_IN[row])
print('hours')
print(238/60)
print('minutes')
print(238-3*60)
print(df.origin_city[row])
print(df.dest_city[row])
print('2hr time zone difference')
print('Calculated arrival time:')
print(14+3+2+1)
print(30-2)
print('Arrival time: ')
print(df.ARR_TIME[row])

Departure Time
14:30:00
Transit time
238.0
hours
3.966666666666667
minutes
58
TUCSON, ARIZONA, USA               
ATLANTA, GEORGIA, USA              
2hr time zone difference
Calculated arrival time:
20
28
Arrival time: 
2028.0


## Logical assessment of features

**Features to include in the model**

X values
* 'OP_CARRIER': airline designation
* 'OP_CARRIER_FL_NUM': flight number
* 'day_of_week': flight day of the week
* 'DEST': destination airport code
* 'CRS_DEP_TIME': scheduled departure time 
* 'CRS_ARR_TIME': scheduled arrival time
* 'DISTANCE': flight distance
* 'CANCELLED': flight cancelled
* 'DIVERTED': flight diverted
* 'CARRIER_DELAY_BOOL': arrival delay due to carrier, True/False
* 'WEATHER_DELAY_BOOL': arrival delay due to weather, True/False
* 'NAS_DELAY_BOOL': arrival delay due to NAS, True/False
* 'SECURITY_DELAY_BOOL': arrival delay due to security, True/False
* 'LATE_AIRCRAFT_DELAY_BOOL': arrival delay due to aircraft, True/False

Y values
* 'ARR_DELAY': arrival time delay


**Features not to inlcude in the model due to irrelevance**
* 'Unnamed: 0': extra index column
* 'ORIGIN': departure city, filtered for TUS only
* 'origin_city': departure city calculated from ORIGIN
* 'dest_city': destination city calculated from DEST
* 'CANCELLATION_CODE': reason for cancellation
* 'CRS_ELAPSED_TIME': scheduled elapsed time

**Features not to inlcude in the model due to calculations**
* 'FL_DATE': flight date provides day_of_week
* 'DEP_TIME': actual departure time, CRS_DEP_TIME + DEP_DELAY
* 'WHEELS_OFF': actual wheels off time, DEP_TIME + TAXI_OUT
* 'WHEELS_ON': actual wheels on time, ARR_TIME - TAXI_IN
* 'ACTUAL_ELAPSED_TIME': TAXI_OUT + TAXI_IN + AIR_TIME
* 'ARR_TIME': actual arrival time, DEP_TIME + ACTUAL_ELAPSED_TIME + time zone difference

**Features not to include in the model due to they directly reveal Y values**
* 'DEP_DELAY': departure delay time
* 'TAXI_OUT': calculated time spent in taxi between departure time and wheels off
* 'TAXI_IN': calculated time spent in taxi between wheels on and arrival time
* 'AIR_TIME': calculated time spent in the air
* 'CARRIER_DELAY': arrival delay time due to carrier
* 'WEATHER_DELAY': arrival delay time due to weather
* 'NAS_DELAY': arrival delay time due to NAS
* 'SECURITY_DELAY': arrival delay time due to security
* 'LATE_AIRCRAFT_DELAY': arrival delay time due to aircraft


In [5]:
correlation_df = df.corr()
correlation_df

,Unnamed: 0,OP_CARRIER_FL_NUM,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
Unnamed: 0,1.000000,0.211644,0.106204,0.022357,0.089115,0.105379,-0.005279,-0.035274,-0.006429,0.028075,0.018811,-0.001159,-0.024935,-0.021835,-0.031809,-0.097655,-0.015859,-0.000987,0.047359,0.009924,0.042223
OP_CARRIER_FL_NUM,0.211644,1.000000,-0.066942,0.028463,0.221331,-0.057354,-0.179910,-0.001091,-0.179848,0.043284,0.020024,-0.004954,-0.192408,-0.173218,-0.215305,-0.285792,0.012009,0.012948,0.063408,0.014968,0.017860
DEP_TIME,0.106204,-0.066942,1.000000,0.189506,0.008086,0.995048,0.906444,-0.044851,0.904801,0.173721,-0.179308,0.006516,-0.164160,-0.121590,-0.119589,-0.159148,0.045733,0.027171,0.029890,-0.006977,0.231984
DEP_DELAY,0.022357,0.028463,0.189506,1.000000,0.070294,0.185716,0.149272,-0.014808,0.145887,0.967141,-0.007829,0.001683,0.015882,0.021861,0.013313,0.005925,0.722701,0.113030,0.448543,0.005421,0.502338
TAXI_OUT,0.089115,0.221331,0.008086,0.070294,1.000000,0.038023,0.030309,0.102668,0.033445,0.189340,-0.149419,0.010805,0.033794,0.190100,0.015761,-0.047767,0.054090,0.042494,0.201561,-0.007327,0.030477
WHEELS_OFF,0.105379,-0.057354,0.995048,0.185716,0.038023,1.000000,0.910051,-0.040460,0.908644,0.175303,-0.200951,0.006617,-0.165642,-0.115537,-0.118958,-0.163005,0.041565,0.028096,0.036134,-0.006813,0.233858
WHEELS_ON,-0.005279,-0.179910,0.906444,0.149272,0.030309,0.910051,1.000000,-0.024187,0.998125,0.136376,-0.242553,0.024856,0.151322,0.194263,0.200246,0.171799,0.029954,0.022198,0.015854,-0.009391,0.202393
TAXI_IN,-0.035274,-0.001091,-0.044851,-0.014808,0.102668,-0.040460,-0.024187,1.000000,-0.000692,0.098445,-0.105210,0.008250,0.019305,0.156958,-0.011295,-0.027078,-0.003152,0.049173,0.136036,0.010858,-0.012946
ARR_TIME,-0.006429,-0.179848,0.904801,0.145887,0.033445,0.908644,0.998125,-0.000692,1.000000,0.135582,-0.245633,0.024592,0.150378,0.196618,0.198473,0.169644,0.029543,0.024233,0.012687,-0.008571,0.201635
ARR_DELAY,0.028075,0.043284,0.173721,0.967141,0.189340,0.175303,0.136376,0.098445,0.135582,1.000000,-0.006279,-0.003887,-0.013342,0.056364,0.011371,-0.021997,0.697762,0.124190,0.498885,0.006949,0.491033
